In [ ]:
!pip install ultralytics==8.3.191

In [ ]:
from ultralytics import YOLO
from IPython.display import display
from PIL import Image
from datetime import datetime
import torch
import cv2

print(torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("Current device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

colors = [
    (255, 0, 0),
    (255, 165, 0),
    (255, 255, 0),
    (0, 255, 0),
    (0, 0, 255),
    (75, 0, 130),
    (128, 0, 128),
    (255, 192, 203),
    (165, 42, 42),
    (0, 255, 255),
    (128, 128, 128)
]


def show_pred(model, img_path, conf=0.1):
    original_img = cv2.imread(img_path)
    original_img_rgb = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)

    results = model.predict(source=img_path, conf=conf)
    result = results[0]

    class_names = model.names

    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        class_id = int(box.cls[0])
        confidence = float(box.conf[0])

        color = colors[class_id % len(colors)]

        cv2.rectangle(original_img_rgb, (x1, y1), (x2, y2), color, 2)

        label = f"{class_names[class_id]}: {confidence:.2f}"

        (text_width, text_height), baseline = cv2.getTextSize(
            label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1
        )

        cv2.rectangle(
            original_img_rgb,
            (x1, y1 - text_height - 5),
            (x1 + text_width, y1),
            color,
            -1
        )

        cv2.putText(
            original_img_rgb,
            label,
            (x1, y1 - 5),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 255, 255),
            1
        )

    # display(Image.open("dataset/03.png"))
    display(Image.fromarray(original_img_rgb))

In [ ]:
yolo11n = YOLO("yolo11n.pt")

show_pred(yolo11n, "dataset/03.png")

In [ ]:
yolo11n.train(
    data="dataset/dataset.yaml",
    epochs=30,
    imgsz=640,
    batch=16
)

new_model_name = f"{datetime.now().strftime('%Y%m%d_%H%M%S')}.pt"

yolo11n.save(new_model_name)

In [ ]:
new_model = YOLO(new_model_name)
show_pred(new_model, "dataset/03.png", 0.02)
show_pred(new_model, "dataset/04.png", 0.02)
show_pred(new_model, "dataset/14.png", 0.02)